In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

# Load Data

In [12]:
file = 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures\\return-01-within-day.tsv'
data = read_tsv(file)
features = ['return', 'return_1', 'return_2','zscore_36_return', 'zscore_288_return', 'pma36', 'ma_12_108','ma_36_288', 
            'ac36', 'ac288', 'zscore_36_volume', 'zscore_288_volume','zscore_2016_volume', 
            'vol36', 'vol288', 'vol2016', 'dv_36_288']

features = ['vol2016', 'ac288', 'vol288', 'ac36', 'return', 'dv_36_288']
data_manager = CurrencyDataManager(data['close'], data[features])
wallet = {'usdt': 1000}
contract_pair = ContractPair.new('usdt', 'btc')

# Load Model

In [13]:
from sklearn.externals import joblib
from modeling.gridsearch import GridSearch

path = "D:\\Dropbox\\My work\\krypl-project\\models\\return-01-within-day"
clf = joblib.load(f'{path}\\bayes_selected_01.pkl')

# Implementation

In [14]:
from modeling.strategy import Strategy

class HoldStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size):
        super().__init__(exchange, data_manager, contract_pair, trade_size, 0., 0.)    
    
    def trade(self):
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(1)
            
            if history.shape[0] == 0:
                self.buy(price)
            else:
                continue
        
        if self.opened:
            self.sell_all(price)

In [15]:
from modeling.strategy import Strategy


class MachineLearningStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit, clf, max_hold_time):
        super().__init__(exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit)
        self.history_len = 1
        self.max_hold_time = max_hold_time
        self.clf = clf
    
    def should_buy(self, history):
        if self.trade_size >= self.exchange.wallet[contract_pair['priceContract']] * 1.1:
            return False
        
        prediction = self.clf.predict(history)[0]
        return bool(prediction)
    
    def should_sell(self, hold):
        return hold >= self.max_hold_time
    
    
    def trade(self):
        after_sell = False
        hold = 0
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(self.history_len)
            
            if history.shape[0] == 0:
                continue
            
            if not self.opened and self.should_buy(history):
                self.buy(price)
                price_bought = price
                hold = 0
            elif self.opened and self.is_risky(price_bought, price):
                self.sell_all(price)
                after_sell = True
                hold = 0
            elif self.opened and self.is_target_satisfied(price_bought, price):
                self.sell_all(price)
                after_sell = True
                hold = 0
            elif self.opened and self.should_sell(hold):
                self.sell_all(price)
                after_sell = True
                hold = 0
            
            if self.opened:
                hold += 1
        
        if self.opened:
            self.sell_all(price)

In [16]:
def hold_stats(data_manager_p):
    data_manager = deepcopy(data_manager_p)
    exchange = BackTestExchange(data_manager, deepcopy(wallet), 0.0025)
    strategy = HoldStrategy(exchange, data_manager, contract_pair, 100)
    strategy.trade()
    return strategy.stats('usdt').report()

# Trading

In [17]:
hold_stats_train = hold_stats(data_manager)
hold_stats_train

,usdt
startAmount,"1,000.0000"
numberOfTrades,1.0000
totalProfit,581.6692
avgProfit,581.6692
winPercentage,100.0000
avgWinTrade,581.6692
avgLossTrade,0.0000
profitFactor,inf
maxDrawdown,0.0000


In [18]:
DAY = (60 / 5) * 24
dm = deepcopy(data_manager)
exchange = BackTestExchange(dm, deepcopy(wallet), 0.0025)
strategy = MachineLearningStrategy(exchange, dm, contract_pair, 100, willing_loss=1, target_profit=0.01, clf=clf, max_hold_time=DAY)
strategy.trade()
r = strategy.stats('usdt').report()
r

,usdt
startAmount,"1,000.0000"
numberOfTrades,951.0000
totalProfit,-188.4344
avgProfit,-0.1981
winPercentage,75.1840
avgWinTrade,1.2933
avgLossTrade,-4.7168
profitFactor,0.8307
maxDrawdown,20.1666


# Look on transactions

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
%matplotlib notebook
plot_transactions(data.copy(), strategy.exchange.transactions[:])